# Load the Dataset

In [1]:
import pdfplumber
import glob
import json
import re

In [31]:
pdf_path = '../data/Richard Hendriks.pdf' # as agreed in whatsapp

pdf_text = ''
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            pdf_text += page_text + '\n'

print(pdf_text)

4/19/25, 10:19 PM Richard Hendriks
richard.hendriks@mail.com
Richard Hendriks
http://richardhendricks.example.com
San Francisco, California
SUMMARY
Richard hails from Tulsa. He has earned degrees from the University of Oklahoma and Stanford. (Go Sooners and
Cardinal!) Before starting Pied Piper, he worked for Hooli as a part time software developer. While his work focuses on
applied information theory, mostly optimizing lossless compression schema of both the length-limited and adaptive
variants, his non-work interests range widely, everything from quantum computing to chaos theory. He could tell you about
it, but THAT would NOT be a “length-limited” conversation!
SKILLS AND TECH
Web Development: HTML, CSS, Javascript.
Compression: Mpeg, MP4, GIF.
EXPERIENCE
Pied Piper CEO/President Dec 2013 — Dec 2014
Build an algorithm for artist to detect if their music was violating copy right infringement laws
Successfully won Techcrunch Disrupt
Optimized an algorithm that holds the current world 

# Use regex to break the text into sections
## LLM can be used but would be too slow for the demo

In [32]:
# Extract the 'SUMMARY' section from the resume text by grabbing everything from the beginning up to 'SKILLS AND TECH'
basics_extracted = pdf_text.split('SKILLS AND TECH')[0]
print(basics_extracted)

4/19/25, 10:19 PM Richard Hendriks
richard.hendriks@mail.com
Richard Hendriks
http://richardhendricks.example.com
San Francisco, California
SUMMARY
Richard hails from Tulsa. He has earned degrees from the University of Oklahoma and Stanford. (Go Sooners and
Cardinal!) Before starting Pied Piper, he worked for Hooli as a part time software developer. While his work focuses on
applied information theory, mostly optimizing lossless compression schema of both the length-limited and adaptive
variants, his non-work interests range widely, everything from quantum computing to chaos theory. He could tell you about
it, but THAT would NOT be a “length-limited” conversation!



In [33]:
# Extract the 'SKILLS AND TECH' section from the resume text by grabbing everything from 'SKILLS AND TECH' to 'EXPERIENCE'
skills_extracted = pdf_text.split('EXPERIENCE')[0].split('SKILLS AND TECH')[1]
print(skills_extracted)


Web Development: HTML, CSS, Javascript.
Compression: Mpeg, MP4, GIF.



In [34]:
# Extract the 'EXPERIENCE' section from the resume text by grabbing everything from 'EXPERIENCE' to 'VOLUNTEERING'
work_extracted = pdf_text.split('VOLUNTEERING')[0].split('EXPERIENCE')[1]
print(work_extracted)


Pied Piper CEO/President Dec 2013 — Dec 2014
Build an algorithm for artist to detect if their music was violating copy right infringement laws
Successfully won Techcrunch Disrupt
Optimized an algorithm that holds the current world record for Weisman Scores



In [35]:
# Extract the 'PROJECTS' section from the resume text by grabbing everything from 'PROJECTS' to 'EDUCATION'
projects_extracted = pdf_text.split('EDUCATION')[0].split('PROJECTS')[1]
print(projects_extracted)


Miss Direction: Won award at AIHacks 2016. Built by all women team of newbie programmers. Using modern
technologies such as GoogleMaps, Chrome Extension and Javascript.



In [36]:
# Extract the 'EDUCATION' section from the resume text by grabbing everything from 'EDUCATION' to 'PUBLICATIONS'
education_extracted = pdf_text.split('PUBLICATIONS')[0].split('EDUCATION')[1]
print(education_extracted)


University of Oklahoma Jun 2011 — Jan 2014
Bachelor - Information Technology, GPA: 4.0



## LANGCHAIN

### Using the new LCEL Architecture from LangChain.
LangChain recommends using LCEL (LangChain Expression Language) over Chains. 

Now we create the retriever object, the responsible to return the data contained in the ChromaDB Database.

In [3]:
!pip install huggingface_hub==0.26.0
!pip install --upgrade huggingface_hub

  Using cached huggingface_hub-0.26.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.26.0-py3-none-any.whl (447 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.51.0 requires huggingface-hub<1.0,>=0.30.0, but you have huggingface-hub 0.26.0 which is incompatible.
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.0
    Uninstalling huggingface-hub-0.26.0:
      Successfully uninstalled huggingface-hub-0.26.0


In [4]:
from getpass import getpass
hf_key = getpass("Hugging Face Key: ")

In [5]:
!huggingface-cli login --token $hf_key

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `langchain` has been saved to /Users/ebengunadi/.cache/huggingface/stored_tokens
Your token has been saved to /Users/ebengunadi/.cache/huggingface/token
Login successful.
The current active token is: `langchain`


## Importing LangChain Libraries.

In [6]:
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/Users/ebengunadi/anaconda3/envs/grids-resume/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch
from torch import cuda

In [9]:
#In a MAC Silicon the device must be 'mps'
device = torch.device('mps') #to use with MAC Silicon
# device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [10]:
device

device(type='mps')

##Load the Model .

In [11]:
#You can try with any llama model, but you will need more GPU and memory as you
#increase the size of the model.
# model_id = "meta-llama/Llama-3.2-1B-Instruct" # Agreed per the 3/11/25 meeting, but not powerful enough!
model_id = "meta-llama/Llama-3.2-3B-Instruct" 

In [ ]:
# !pip install --upgrade transformers 
# !pip install accelerate

In [12]:
from accelerate import init_empty_weights

In [13]:
%%time

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    token=hf_key
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    token=hf_key
)
model.eval()
print(f"Model loaded on {device}")


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.19s/it]


Model loaded on mps
CPU times: user 3.29 s, sys: 3.13 s, total: 6.42 s
Wall time: 10.9 s


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_aut_token=hf_key)


In [15]:
# Set up the HuggingFace pipeline with your desired settings.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256, # ideally would be 1024, but takes too long given the resource constraints
    temperature=0,         # Deterministic output
    top_p=1.0,             # Allow full probability distribution
    do_sample=False,       # Greedy decoding
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.0,  # Lowered if necessary
    return_full_text=False,
    device_map="auto"
)
hf_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use mps


In [38]:
# Define JSON schemas for conversion.
basics_schema = '''{
    "name": "",
    "label": "",
    "image": "",
    "email": "",
    "phone": "",
    "url": "",
    "summary": "",
    "location": {
        "address": "",
        "postalCode": "",
        "city": "",
        "countryCode": "",
        "region": ""
    },
    "profiles": [
        {
            "network": "",
            "username": "",
            "url": ""
        }
    ]
}'''

work_schema = '''[
    {
        "name": "",
        "location": "",
        "description": "",
        "position": "",
        "url": "",
        "startDate": "",
        "endDate": "",
        "summary": "",
        "highlights": []
    }
]'''

education_schema = '''[
    {
        "institution": "",
        "url": "",
        "area": "",
        "studyType": "",
        "startDate": "",
        "endDate": "",
        "score": "",
        "courses": []
    }
]'''

skills_schema = '''[
    {
        "name": "",
        "level": "",
        "keywords": []
    }
]'''

projects_schema = '''[
    {
        "name": "",
        "description": "",
        "highlights": [],
        "keywords": [],
        "startDate": "",
        "endDate": "",
        "url": "",
        "roles": [],
        "entity": "",
        "type": ""
    }
]'''

In [52]:
# --- Optimized Conversion Prompts ---
# These prompts convert the extracted raw text into the desired JSON format without any extra commentary.
basics_conversion_prompt = (
    "Convert the extracted text into a JSON object that strictly adheres to the provided JSON schema. "
    "Use only the information present in the text; for any field not mentioned, set its value to an empty string. "
    "Output only the JSON object with no extra commentary or code.\n\n"
    "Schema: {schema}\n\n"
    "Extracted Text:\n{text}"
)

work_conversion_prompt = (
    "Convert the extracted text into a JSON array that strictly adheres to the provided JSON schema. "
    "Use only the information present in the text; for any field not mentioned, set its value to an empty string. "
    "Output only the JSON array with no extra commentary or code.\n"
    "### Schema:\n{schema}\n"
    "### Extracted Text:\n{text}"
)

education_conversion_prompt = (
    "Convert the extracted text into a JSON array that strictly adheres to the provided JSON schema. "
    "Use only the information present in the text; for any field not mentioned, set its value to an empty string. "
    "Output only the JSON array with no extra commentary or code.\n\n"
    "Schema: {schema}\n\n"
    "Extracted Text:\n{text}"
)

skills_conversion_prompt = (
    "Convert the extracted text into a JSON array that strictly adheres to the provided JSON schema. "
    "Use only the information present in the text; for any field not mentioned, set its value to an empty string. "
    "Output only the JSON array with no extra commentary or code.\n\n"
    "Schema: {schema}\n\n"
    "Extracted Text:\n{text}"
)

projects_conversion_prompt = (
    "Convert the extracted text into a JSON array that strictly adheres to the provided JSON schema. "
    "Use only the information present in the text; for any field not mentioned, set its value to an empty string. "
    "Output only the JSON array with no extra commentary or code.\n\n"
    "Schema: {schema}\n\n"
    "Extracted Text:\n{text}"
)

In [53]:
# Create conversion chains.
basics_conversion_chain = (
    {"text": RunnablePassthrough(), "schema": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(basics_conversion_prompt)
    | hf_llm
    | StrOutputParser()
)
work_conversion_chain = (
    {"text": RunnablePassthrough(), "schema": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(work_conversion_prompt)
    | hf_llm
    | StrOutputParser()
)
education_conversion_chain = (
    {"text": RunnablePassthrough(), "schema": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(education_conversion_prompt)
    | hf_llm
    | StrOutputParser()
)
skills_conversion_chain = (
    {"text": RunnablePassthrough(), "schema": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(skills_conversion_prompt)
    | hf_llm
    | StrOutputParser()
)
projects_conversion_chain = (
    {"text": RunnablePassthrough(), "schema": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(projects_conversion_prompt)
    | hf_llm
    | StrOutputParser()
)

In [41]:
%%time

basics_json_response = basics_conversion_chain.invoke({"text": basics_extracted, "schema": basics_schema})

print(basics_json_response)

/Users/ebengunadi/anaconda3/envs/grids-resume/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 

JSON Object:
{
    "name": "",
    "label": "",
    "image": "",
    "email": "richard.hendriks@mail.com",
    "phone": "",
    "url": "http://richardhendricks.example.com",
    "summary": "Richard hails from Tulsa. He has earned degrees from the University of Oklahoma and Stanford. (Go Sooners and\nCardinal!) Before starting Pied Piper, he worked for Hooli as a part time software developer. While his work focuses on\napplied information theory, mostly optimizing lossless compression schema of both the length-limited and adaptive\nvariants, his non-work interests range widely, everything from quantum computing to chaos theory. He could tell you about\nit, but THAT would NOT be a “length-limited” conversation!",
    "location": {
        "address": "",
        "postalCode": "",
        "city": "San Francisco, California",
        "countryCode": "",
        "region": ""
    },
    "profiles": [
        {
            "network": "",
            "username": "",
            "url": ""
    

In [42]:
def extract_json_object(response: str) -> str:
    """
    Extracts the JSON substring from the LLM response.
    """
    match = re.search(r'(\{.*\})', response, re.DOTALL)
    if match:
        return match.group(1)
    return response


In [43]:
basics_json_parsed = json.loads(extract_json_object(basics_json_response))
basics_json_parsed


{'name': '',
 'label': '',
 'image': '',
 'email': 'richard.hendriks@mail.com',
 'phone': '',
 'url': 'http://richardhendricks.example.com',
 'summary': 'Richard hails from Tulsa. He has earned degrees from the University of Oklahoma and Stanford. (Go Sooners and\nCardinal!) Before starting Pied Piper, he worked for Hooli as a part time software developer. While his work focuses on\napplied information theory, mostly optimizing lossless compression schema of both the length-limited and adaptive\nvariants, his non-work interests range widely, everything from quantum computing to chaos theory. He could tell you about\nit, but THAT would NOT be a “length-limited” conversation!',
 'location': {'address': '',
  'postalCode': '',
  'city': 'San Francisco, California',
  'countryCode': '',
  'region': ''},
 'profiles': [{'network': '', 'username': '', 'url': ''}]}

In [55]:
print(work_extracted)


Pied Piper CEO/President Dec 2013 — Dec 2014
Build an algorithm for artist to detect if their music was violating copy right infringement laws
Successfully won Techcrunch Disrupt
Optimized an algorithm that holds the current world record for Weisman Scores



In [56]:
print(work_schema)

[
    {
        "name": "",
        "location": "",
        "description": "",
        "position": "",
        "url": "",
        "startDate": "",
        "endDate": "",
        "summary": "",
        "highlights": []
    }
]


In [54]:
%%time

work_json_response = work_conversion_chain.invoke({"text": work_extracted, "schema": work_schema})

print(work_json_response)

/Users/ebengunadi/anaconda3/envs/grids-resume/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 

```json
[
  {
    "name": "",
    "location": "",
    "description": "",
    "position": "",
    "url": "",
    "startDate": "",
    "endDate": "",
    "summary": "",
    "highlights": []
  }
]
``` 

Note: The provided schema and extracted text are identical, which means the output will be the same as the input. However, in a real-world scenario, you would need to parse the extracted text and create a new JSON array based on the schema.
CPU times: user 13.6 s, sys: 2min 14s, total: 2min 27s
Wall time: 6min 29s


In [89]:
def extract_first_json_array(response: str) -> str:
    """
    Extracts the JSON array substring from the LLM response.
    Due to the 256 max token limit, just return the first object in the array and add a closing bracket.
    """
    match = re.search(r'(\[.*?\})', response, re.DOTALL)
    if match:
        return match.group(1) + ']'
    return response
    

In [90]:
work_json_parsed = json.loads(extract_first_json_array(work_json_response))
work_json_parsed


[{'name': 'Information Technology Supervisor',
  'location': 'City, State',
  'description': '',
  'position': 'Supervise up to 10 personnel at one time, delegating tasks, conducting performance evaluations and providing corrective counseling as necessary. Train personnel in the set-up and proper use of IT related equipment while adhering to all policies and procedures. Responsible for the inventory of over $1 million worth of network communications equipment. Tasked by President of the United States to act as supervisor and maintain signal communications for Fort Bragg army base.',
  'url': '',
  'startDate': '01/2011',
  'endDate': '05/2014',
  'summary': '',
  'highlights': []}]

In [103]:
%%time

# if the extracted text startsWith "No.*section found" is in the extracted text, just return the schema
# else run the conversion chain

if re.match(r"^No\s+\w+\s+section\s+found\.?", education_extracted.strip(), re.IGNORECASE) is not None:
    education_json_response = education_schema
else:
    education_json_response = education_conversion_chain.invoke({"text": education_extracted, "schema": education_schema})

print(education_json_response)

/Users/ebengunadi/anaconda3/envs/grids-resume/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 Do not include the course numbers.

```json
[
  {
    "institution": "Microsoft",
    "url": "",
    "area": "",
    "studyType": "",
    "startDate": "",
    "endDate": "",
    "score": "",
    "courses": [
      "Information Technology (Network Communications)",
      "IT Network and Cisco Routing"
    ]
  },
  {
    "institution": "Comptia",
    "url": "",
    "area": "",
    "studyType": "",
    "startDate": "",
    "endDate": "",
    "score": "",
    "courses": [
      "Security"
    ]
  },
  {
    "institution": "U.S. Army",
    "url": "",
    "area": "",
    "studyType": "",
    "startDate": "",
    "endDate": "",
    "score": "",
    "courses": [
      "Information Technology (Network Communications)"
    ]
  },
  {
    "institution": "Northwest Florida State College",
    "url": "",
    "area": "",
    "studyType": "",
    "startDate": "",
    "endDate": "",
    "score": "",
    "courses": [
      "Radiography"
    ]
  },
  {
    "institution": "
CPU times: user 18.2 s, sys: 

In [104]:
education_json_parsed = json.loads(extract_first_json_array(education_json_response))
education_json_parsed


[{'institution': 'Microsoft',
  'url': '',
  'area': '',
  'studyType': '',
  'startDate': '',
  'endDate': '',
  'score': '',
  'courses': ['Information Technology (Network Communications)',
   'IT Network and Cisco Routing']}]

In [105]:
%%time

# if the extracted text startsWith "No.*section found" is in the extracted text, just return the schema
# else run the conversion chain

if re.match(r"^No\s+\w+\s+section\s+found\.?", skills_extracted.strip(), re.IGNORECASE) is not None:
    skills_json_response = skills_schema
else:
    skills_json_response = skills_conversion_chain.invoke({"text": skills_extracted, "schema": skills_schema})

print(skills_json_response)

[
    {
        "name": "",
        "level": "",
        "keywords": []
    }
]
CPU times: user 91 µs, sys: 122 µs, total: 213 µs
Wall time: 428 µs


In [106]:
skills_json_parsed = json.loads(extract_first_json_array(skills_json_response))
skills_json_parsed


[{'name': '', 'level': '', 'keywords': []}]

In [107]:
%%time

# if the extracted text startsWith "No.*section found" is in the extracted text, just return the schema
# else run the conversion chain

if re.match(r"^No\s+\w+\s+section\s+found\.?", projects_extracted.strip(), re.IGNORECASE) is not None:
    projects_json_response = projects_schema
else:
    projects_json_response = projects_conversion_chain.invoke({"text": projects_extracted, "schema": projects_schema})

print(projects_json_response)

[
    {
        "name": "",
        "description": "",
        "highlights": [],
        "keywords": [],
        "startDate": "",
        "endDate": "",
        "url": "",
        "roles": [],
        "entity": "",
        "type": ""
    }
]
CPU times: user 269 µs, sys: 187 µs, total: 456 µs
Wall time: 634 µs


In [108]:
projects_json_parsed = json.loads(extract_first_json_array(projects_json_response))
projects_json_parsed


[{'name': '',
  'description': '',
  'highlights': [],
  'keywords': [],
  'startDate': '',
  'endDate': '',
  'url': '',
  'roles': [],
  'entity': '',
  'type': ''}]

## Put together the final JSON

In [109]:
# put it all together

resume_json = {
    "$schema": "https://raw.githubusercontent.com/jsonresume/resume-schema/v1.0.0/schema.json",
    "basics": basics_json_parsed,
    "work": work_json_parsed,
    "education": education_json_parsed,
    "skills": skills_json_parsed,
    "projects": projects_json_parsed,
    "meta": {
        "canonical": "https://raw.githubusercontent.com/jsonresume/resume-schema/v1.0.0/sample.resume.json",
        "version": "v1.0.0",
        "lastModified": "2017-12-24T15:53:00"
    }
}

resume_json

{'$schema': 'https://raw.githubusercontent.com/jsonresume/resume-schema/v1.0.0/schema.json',
 'basics': {'name': '',
  'label': '',
  'image': '',
  'email': '',
  'phone': '',
  'url': '',
  'summary': '',
  'location': {'address': '',
   'postalCode': '',
   'city': '',
   'countryCode': '',
   'region': ''},
  'profiles': [{'network': '', 'username': '', 'url': ''}]},
 'work': [{'name': 'Information Technology Supervisor',
   'location': 'City, State',
   'description': '',
   'position': 'Supervise up to 10 personnel at one time, delegating tasks, conducting performance evaluations and providing corrective counseling as necessary. Train personnel in the set-up and proper use of IT related equipment while adhering to all policies and procedures. Responsible for the inventory of over $1 million worth of network communications equipment. Tasked by President of the United States to act as supervisor and maintain signal communications for Fort Bragg army base.',
   'url': '',
   'startD

In [111]:
output_file = '../data/INFORMATION-TECHNOLOGY-JSON/33241454.json'
with open(output_file, 'w') as f:
    json.dump(resume_json, f, indent=4)
    print(f"Resume JSON saved to {output_file}")

Resume JSON saved to ../data/INFORMATION-TECHNOLOGY-JSON/33241454.json


## Cleanup



In [ ]:
# !pip install "huggingface_hub[cli]"

In [ ]:
# cleanup huggingface LLMs

# run this command in the terminal
# huggingface-cli delete-cache